In [0]:
!pip install azure.storage.blob #INSTALA BIBLIOTECA PARA CONEXÃO AO BLOB

In [0]:
import pandas as pd 
from azure.storage.blob import BlobServiceClient
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# VARIÁVEIS DE CONEXÃO DO BLOB
STORAGEACCOUNTURL= "https://stgestudos.blob.core.windows.net"
STORAGEACCOUNTKEY= "?????????????????????????????????????????????????????????????????????????????????"
LOCALFILENAME= "orders.csv"
CONTAINERNAME= "pocco-company"
BLOBNAME= "orders.csv"

# CONECTA AO BLOB 
blob_service_client_instance = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)
blob_client_instance = blob_service_client_instance.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)
with open(LOCALFILENAME, "wb") as my_blob:
    blob_data = blob_client_instance.download_blob()
    blob_data.readinto(my_blob)

In [0]:
# CRIA O DATAFRAME DO ARQUIVO DO BLOB.
df = pd.read_csv(LOCALFILENAME, sep=';', index_col=False)

# ALTERA NOME DAS COLUNAS
df.columns=['Region','Country', 'Item_type','Sales_Channel','Order_Priority','Order_Date','Order_ID','Ship_Date','Units_Sold','Unit_Price','Unit_Cost','Total_Revenue','Total_Cost','Total_Profit']

# CRIA UM DATAFRAME DO SPARK (df2) A PARTIR DO DATAFRAME DO PANDAS (df).
df2 = spark.createDataFrame(df)

In [0]:
# CRIA UM DATRAFRAME (df3) CONVERTIDO PARA NOVOS TIPOS (DECIMAl, DATE E INTEGER) E SUBSTITUI VIRGULAS POR PONTOS (sql.types/sql.functions)
df3 = (df2
                      .withColumn('Unit_Price', regexp_replace('Unit_Price', ",", "."))
                      .withColumn('Unit_Cost', regexp_replace('Unit_Cost', ",", "."))
                      .withColumn('Total_Revenue', regexp_replace('Total_Revenue', ",", "."))
                      .withColumn('Total_Cost', regexp_replace('Total_Cost', ",", "."))
                      .withColumn('Total_Profit', regexp_replace('Total_Profit', ",", "."))
                      .withColumn('Ship_Date', to_date('Ship_Date', 'dd/MM/yyyy'))
                      .withColumn('Order_Date', to_date('Order_Date', 'dd/MM/yyyy'))
                      .withColumn('Order_ID', col('Order_ID').cast(IntegerType()))
                      .withColumn('Units_Sold',col('Units_Sold').cast(IntegerType()))
                      .withColumn('Unit_Price',col('Unit_Price').cast(DecimalType(15,2)))
                      .withColumn('Unit_Cost',col('Unit_Cost').cast(DecimalType(15,2)))
                      .withColumn('Total_Revenue',col('Total_Revenue').cast(DecimalType(15,2)))
                      .withColumn('Total_Cost',col('Total_Cost').cast(DecimalType(15,2)))
                      .withColumn('Total_Profit',col('Total_Profit').cast(DecimalType(15,2)))                                             
)

In [0]:
#CONECTA AO SQL COM AS CREDENCIAIS DO AZURE SQL DB E COPIA OS DADOS DO DF3 PARA A TABELA STAGE
df3.write.jdbc(url= "jdbc:sqlserver://sql-estudo.database.windows.net:1433; database=db-estudos; user=????????; password=?????????; encrypt=true; trustServerCertificate=false; hostNameInCertificate=*.database.windows.net; loginTimeout=30; driver=com.microsoft.sqlserver.jdbc.SQLServerDriver", table= "STAGE_frederico_camargo.Sales_Pocco_", mode= "overwrite")